In [ ]:
! pip install bert-for-sequence-classification

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 6.5 MB/s 
     |████████████████████████████████| 7.6 MB 42.8 MB/s 
     |████████████████████████████████| 163 kB 69.6 MB/s 


In [ ]:
import os
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
import json

from transformers import AutoModel, AutoTokenizer

from bert_clf import BertCLF, train_evaluate, predict_metrics, prepare_data_notebook, prepare_dataset
from bert_clf.utils import set_global_seed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/My Drive/UP_w22/PM/task 8/sample/str1_sample_train_sent.csv')
df

Mounted at /content/drive


,Unnamed: 0,post_id,subreddit_id,Sentence,Label,Components
0,0,rpzuhe,t5_2qlaa,Hey guys in new here and just had some questio...,none,{}
1,1,rpzuhe,t5_2qlaa,"After talking to some people, they think I may...",per_exp,"After talking to some people, they think I may..."
2,2,rpzuhe,t5_2qlaa,Ever since almost a year and a half ago I have...,per_exp,Ever since almost a year and a half ago I have...
3,3,rpzuhe,t5_2qlaa,I dont really get heartburn except for on occa...,per_exp,I dont really get heartburn except for on occa...
4,4,rpzuhe,t5_2qlaa,I was told they are all generally normal bacte...,none,{}
...,...,...,...,...,...,...
227,227,smy7j7,t5_2s3g1,Post probiotics\nSo my doc put me on probiotic...,none,{}
228,228,smy7j7,t5_2s3g1,I was on for 2 weeks and I kept feeling worse ...,none,{}
229,229,smy7j7,t5_2s3g1,I made the decision to stop and its been about...,none,{}
230,230,smy7j7,t5_2s3g1,I have a lot more gas then I did prior.,none,{}


In [ ]:
valid = ['claim', 'claim_per_exp', 'per_exp', 'question']
utest = df.loc[(df['Label'].isin(valid))]

In [ ]:
utest

,Unnamed: 0,post_id,subreddit_id,Sentence,Label,Components
1,1,rpzuhe,t5_2qlaa,"After talking to some people, they think I may...",per_exp,"After talking to some people, they think I may..."
2,2,rpzuhe,t5_2qlaa,Ever since almost a year and a half ago I have...,per_exp,Ever since almost a year and a half ago I have...
3,3,rpzuhe,t5_2qlaa,I dont really get heartburn except for on occa...,per_exp,I dont really get heartburn except for on occa...
14,14,rpzuhe,t5_2qlaa,It has caused me to have swollen taste buds an...,per_exp,It has caused me to have swollen taste buds an...
23,23,rpzuhe,t5_2qlaa,Anyone else have these same problems?,question,Anyone else have these same problems
25,25,s5ln4e,t5_2tyg2,No episodes during my stay in psych ward until...,per_exp,Before getting admitted to psych ward I had ep...
26,26,s5ln4e,t5_2tyg2,I got myself admitted due to thinking of harmi...,per_exp,I got myself admitted due to thinking of harmi...
28,28,ptzw1x,t5_2r876,Looking for some positive experiences\nSometim...,claim,Sometimes the CF community in general can get ...
44,44,ptzw1x,t5_2r876,Was there a time someone made a generous accom...,question,Was there a time someone made a generous accom...
60,60,ptzw1x,t5_2r876,Did you have an experience you wouldn't have o...,question,Did you have an experience you wouldn't have o...


In [ ]:
# ['claim', 'claim_per_exp', 'per_exp', 'question']

utest['Label'] = utest['Label'].str.replace('claim_per_exp','Claim Per Experience')
utest['Label'] = utest['Label'].str.replace('claim','Claim')
utest['Label'] = utest['Label'].str.replace('per_exp','Per Experience')
utest['Label'] = utest['Label'].str.replace('question','Question')
utest

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

,Unnamed: 0,post_id,subreddit_id,Sentence,Label,Components
1,1,rpzuhe,t5_2qlaa,"After talking to some people, they think I may...",Per Experience,"After talking to some people, they think I may..."
2,2,rpzuhe,t5_2qlaa,Ever since almost a year and a half ago I have...,Per Experience,Ever since almost a year and a half ago I have...
3,3,rpzuhe,t5_2qlaa,I dont really get heartburn except for on occa...,Per Experience,I dont really get heartburn except for on occa...
14,14,rpzuhe,t5_2qlaa,It has caused me to have swollen taste buds an...,Per Experience,It has caused me to have swollen taste buds an...
23,23,rpzuhe,t5_2qlaa,Anyone else have these same problems?,Question,Anyone else have these same problems
25,25,s5ln4e,t5_2tyg2,No episodes during my stay in psych ward until...,Per Experience,Before getting admitted to psych ward I had ep...
26,26,s5ln4e,t5_2tyg2,I got myself admitted due to thinking of harmi...,Per Experience,I got myself admitted due to thinking of harmi...
28,28,ptzw1x,t5_2r876,Looking for some positive experiences\nSometim...,Claim,Sometimes the CF community in general can get ...
44,44,ptzw1x,t5_2r876,Was there a time someone made a generous accom...,Question,Was there a time someone made a generous accom...
60,60,ptzw1x,t5_2r876,Did you have an experience you wouldn't have o...,Question,Did you have an experience you wouldn't have o...


In [ ]:
df_train = pd.DataFrame(columns=['Sentence', 'Label'])
df_test = pd.DataFrame(columns=['Sentence', 'Label'])
df_value = pd.DataFrame(columns=['Sentence', 'Label'])
split = int(232*0.8)
for index, row in df.iterrows():
  if index < split:
    df_train = df_train.append({'Sentence': row['Sentence'], 'Label': row["Label"]},ignore_index=True)

  if index >= split:
    df_test = df_test.append({'Sentence': row['Sentence'], 'Label': row["Label"]},ignore_index=True)

  df_value = df_value.append({'Sentence': row['Sentence'], 'Label': row["Label"]},ignore_index=True)


print(df_train.shape, df_value.shape, df_test.shape)

(185, 2) (232, 2) (47, 2)


In [ ]:
config = dict(
    transformer_model = dict(
        model = "roberta-base",
        path_to_state_dict = False,
        device = 'cuda',
        dropout = 0.2,
        learning_rate = 2e-6,
        batch_size = 16,
        shuffle = True,
        maxlen = 128,
    ),
    data = dict(
        train_data_path = df_train,
        test_data_path = df_value,
        text_column = "Sentence",
        target_column = "Label",
        random_state = 20,
        test_size = 0.3,
        stratify=True
    ),
    training = dict (
    save_state_dict = False, # if False the model will be saved using torch.save()
        # and should be loaded like this: model = torch.load()
        # you will have to install the library to do so
    early_stopping = True,
    delta = 0.001,
    patience = 7,
    num_epochs = 2,
    average_f1 = 'macro',
    other_metrics = ['micro', 'weighted'],
    output_dir = "../results/",
    class_weight = True
    )
)

In [ ]:
set_global_seed(seed=config['data']['random_state'])
os.makedirs(config['training']['output_dir'], exist_ok=True)

In [ ]:
device = torch.device(config['transformer_model']['device'])
tokenizer = AutoTokenizer.from_pretrained(
        pretrained_model_name_or_path=config['transformer_model']["model"]
    )
model_bert = AutoModel.from_pretrained(
    pretrained_model_name_or_path=config['transformer_model']["model"]
).to(device)

#for param in model_bert.parameters():
    #param.requires_grad = False

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
id2label, train_texts, valid_texts, train_targets, valid_targets = prepare_data_notebook(
    config=config, train_df = df_train, test_df = df_value
)

In [ ]:
model = BertCLF(
    pretrained_model=model_bert,
    tokenizer=tokenizer,
    id2label=id2label,
    dropout=config['transformer_model']['dropout'],
    device=device     
    )

In [ ]:
model = model.to(device)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=float(config['transformer_model']['learning_rate']))
criterion = nn.NLLLoss()

training_generator, valid_generator = prepare_dataset(
    tokenizer=tokenizer,
    train_texts=train_texts,
    train_targets=train_targets,
    valid_texts=valid_texts,
    valid_targets=valid_targets,
    config=config
)

In [ ]:
model = train_evaluate(
    model=model,
    training_generator=training_generator,
    valid_generator=valid_generator,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=config['training']['num_epochs'],
    average=config['training']['average_f1'],
    config=config
)


==== Epoch 1 out of 2 ====


Evaluating loop: 100%|██████████| 15/15 [00:00<00:00, 17.31it/s]


Train F1: 0.08253065221815221
Eval F1: 0.043400986125444324

Train F1 micro: 0.11226851851851853
Eval F1 micro: 0.07083333333333333

Train F1 weighted: 0.0823682972120472
Eval F1 weighted: 0.015167411994037379

==== Epoch 2 out of 2 ====


Evaluating loop: 100%|██████████| 15/15 [00:00<00:00, 20.46it/s]


Train F1: 0.1575202776979738
Eval F1: 0.04517066085693537

Train F1 micro: 0.24016203703703706
Eval F1 micro: 0.07083333333333333

Train F1 weighted: 0.2799151752046973
Eval F1 weighted: 0.01181917211328976




Computing final metrics...: 100%|██████████| 15/15 [00:00<00:00, 21.16it/s]

               precision    recall  f1-score   support

        claim       0.00      0.00      0.00         1
claim_per_exp       0.00      0.00      0.00         7
         none       0.00      0.00      0.00       193
      per_exp       0.07      1.00      0.14        17
     question       0.00      0.00      0.00        14

     accuracy                           0.07       232
    macro avg       0.01      0.20      0.03       232
 weighted avg       0.01      0.07      0.01       232



In [ ]:
model.to('cpu')

BertCLF(
  (pretrained_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(

In [ ]:
preds = []
for i,j in zip(df_test['Sentence'], df_test['Label']):
    preds.append([model.predict(i), j, i])

In [ ]:
pred = []
for i in preds:
    pred.append(i[0])

true = []
for m in preds:
    true.append(m[1])

In [ ]:
from sklearn.metrics import classification_report
target_names = ['class 4','class 3','class 2','class 1', 'class 0']
print(classification_report(true, pred, target_names=target_names, digits=3))

              precision    recall  f1-score   support

     class 3      0.000     0.000     0.000       4.0
     class 2      0.000     0.000     0.000      38.0
     class 1      0.000     0.000     0.000       0.0
     class 0      0.000     0.000     0.000       5.0

    accuracy                          0.000      47.0
   macro avg      0.000     0.000     0.000      47.0
weighted avg      0.000     0.000     0.000      47.0



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [ ]:
preds = []
for i,j in zip(utest['Sentence'], utest['Label']):
    preds.append([model.predict(i), j, i])

In [ ]:
pred = []
for i in preds:
    pred.append(i[0])

true = []
for m in preds:
    true.append(m[1])

In [ ]:
from sklearn.metrics import classification_report
target_names = ['class 4','class 3','class 2','class 1', 'class 0']
print(classification_report(true, pred, target_names=target_names, digits=3))

              precision    recall  f1-score   support

     class 4      0.000     0.000     0.000       1.0
     class 3      0.000     0.000     0.000       7.0
     class 2      0.000     0.000     0.000      17.0
     class 1      0.000     0.000     0.000      14.0
     class 0      0.000     0.000     0.000       0.0

    accuracy                          0.000      39.0
   macro avg      0.000     0.000     0.000      39.0
weighted avg      0.000     0.000     0.000      39.0



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [ ]:
import numpy as np
dummy = np.ones(3814)

In [ ]:
# baseline for task 2 on Ukraine data 
target_names = ['class 3','class 2','class 1', 'class 0']
print(classification_report(true, dummy, target_names=target_names, digits=3))

NameError: ignored